In [2]:
import numpy as np

In [3]:
"""
parallel_scheduler_no_home_paths_vFull_fixed.py
───────────────────────────────────────────────
• Two-tick MS: gate + hold (ions frozen both).
• ≥1 ion exits site right after hold.
• Single-qubit layers fire ASAP:
      – if last tick is empty, merge;
      – otherwise open a new tick.
• No-home-node paths.
• **Now executes every MS gate in a layer**, not just the first one.
"""

import numpy as np
import networkx as nx
from trap import create_trap_graph

# ── 1) load layers ---------------------------------------------------------
with open("time_steps_raw_fixedphi.txt") as f:
    gt_no_time = [eval(l.strip()) for l in f]

L, Q = len(gt_no_time), 8

# ── 2) trap graph & site sets ---------------------------------------------
G = create_trap_graph()
interaction_sites = {n for n, d in G.nodes(data=True)
                     if d["type"] == "interaction"}
standard_sites = {n for n, d in G.nodes(data=True)
                  if d["type"] == "standard"}

# ── 3) initial positions ---------------------------------------------------
pos_hist = np.zeros((1, Q, 2), dtype=int)
pos_hist[0] = np.array([
    (0, 1), (0, 3), (1, 0), (1, 4),
    (3, 0), (3, 4), (4, 1), (4, 3)
])
home_pos      = {q: tuple(pos_hist[0, q]) for q in range(Q)}
initial_nodes = set(home_pos.values())

# ── 4) timeline helpers ----------------------------------------------------
scheduled_ops: list[list[tuple]] = []

def append_tick(ops=None):
    if ops is None:
        ops = []
    scheduled_ops.append(ops)
    global pos_hist
    pos_hist = np.concatenate([pos_hist, pos_hist[-1:]], axis=0)

# ── 5) one-hop parallel router --------------------------------------------
def hop(cur, goal):
    if cur == goal:
        return None
    def w(u, v, data):
        return 1e6 if v in initial_nodes and v != goal else 1
    return nx.dijkstra_path(G, cur, goal, weight=w)[1]

def route_parallel(goals):
    while True:
        active = {q for q, g in goals.items()
                  if tuple(pos_hist[-1, q]) != g}
        if not active:
            break
        prop = {q: hop(tuple(pos_hist[-1, q]), goals[q]) for q in active}
        occ  = {tuple(pos_hist[-1, r]): r for r in range(Q)}
        moves = {}
        # duplicates → lowest index wins
        dest_map = {}
        for q, d in prop.items():
            if d:
                dest_map.setdefault(d, []).append(q)
        for dst, qs in dest_map.items():
            qs.sort()
            if dst in occ and occ[dst] not in prop:
                continue
            moves[qs[0]] = dst
        # prevent swaps
        for q, d in list(moves.items()):
            p = next((p for p, dp in moves.items()
                      if dp == tuple(pos_hist[-1, q]) and p != q), None)
            if p and d == tuple(pos_hist[-1, p]):
                moves.pop(max(q, p))
        if not moves:
            q = min(active)
            moves[q] = prop[q]
        append_tick()
        for q, d in moves.items():
            pos_hist[-1, q] = d

# ── 6) scheduler loop ------------------------------------------------------
for layer in range(L):
    ops_layer = gt_no_time[layer]

    # look-ahead MS qubits for reuse set
    if layer + 1 < L:
        next_ms = [g for g in gt_no_time[layer+1] if g[0] == "MS"]
        next_ms_qubits = {q for g in next_ms for q in g[2]}
    else:
        next_ms_qubits = set()

    # —— Execute all MS gates in this layer ——
    ms_gates = [g for g in ops_layer if g[0] == "MS"]
    for _, angle, (q1, q2) in ms_gates:
        # nearest interaction site
        target = min(
            interaction_sites,
            key=lambda s: sum(abs(s[0]-pos_hist[-1,q][0]) +
                              abs(s[1]-pos_hist[-1,q][1])
                              for q in (q1, q2))
        )
        # load both qubits
        route_parallel({q1: target, q2: target})
        # perform MS (sorted pair) immediately when second arrives
        pair = tuple(sorted((q1, q2)))
        scheduled_ops[-1].append(("MS", angle, pair))
        # hold tick
        append_tick()
        # unload those not reused
        to_unload = {q1, q2} - next_ms_qubits
        if to_unload:
            route_parallel({q: home_pos[q] for q in to_unload})

    # —— Single-qubit rotations (ASAP) ——
    sq_ops = [(n, a, q) for n, a, q in ops_layer if n in ("RX", "RY")]
    if sq_ops:
        # exit interaction site if needed
        exits, occ = {}, set(map(tuple, pos_hist[-1]))
        for _, _, q in sq_ops:
            here = tuple(pos_hist[-1, q])
            if here in interaction_sites:
                for nb in G.neighbors(here):
                    if nb in standard_sites and nb not in occ:
                        exits[q] = nb
                        occ.add(nb)
                        break
        if exits:
            route_parallel(exits)
        # merge or new tick
        if scheduled_ops and not scheduled_ops[-1]:
            scheduled_ops[-1].extend(sq_ops)
        else:
            append_tick(sq_ops)
        # return non-reused
        back = {q: home_pos[q] for _, _, q in sq_ops if q not in next_ms_qubits}
        if back:
            route_parallel(back)

    # —— Idle qubits → home ——
    busy = {q for _, _, q in sq_ops}
    busy |= {q for g in ms_gates for q in g[2]}
    idle = {q: home_pos[q] for q in range(Q)
            if q not in busy and tuple(pos_hist[-1, q]) != home_pos[q]}
    if idle:
        route_parallel(idle)

# final cleanup
final = {q: home_pos[q] for q in range(Q)
         if tuple(pos_hist[-1, q]) != home_pos[q]}
if final:
    route_parallel(final)

print("Total ticks:", len(scheduled_ops))
print("pos_hist.shape:", pos_hist.shape)
print("scheduled_ops (non-empty):")
for t, ops in enumerate(scheduled_ops):
    if ops:
        print(f"t={t}: {ops}")


Total ticks: 437
pos_hist.shape: (438, 8, 2)
scheduled_ops (non-empty):
t=0: [('RY', 1.5707963267948966, 6), ('RX', 0.6154797086703869, 7)]
t=1: [('RX', 1.5707963267948966, 6), ('RY', 1.570796326794897, 7)]
t=2: [('RX', -1.5707963267948966, 7)]
t=5: [('MS', 1.5707963267948966, (6, 7))]
t=9: [('RY', 3.1415926535897927, 6), ('RX', -1.5707963267948966, 7)]
t=10: [('RY', 0.7853981633974495, 7)]
t=13: [('MS', 1.5707963267948966, (6, 7))]
t=17: [('RX', 0.7853981633974492, 6), ('RY', 1.5707963267948963, 7)]
t=18: [('RY', 1.5707963267948963, 6), ('RX', 0.562617536425785, 7)]
t=20: [('MS', 1.5707963267948966, (5, 7))]
t=22: [('RX', -2.356194490192345, 6)]
t=23: [('RY', 0.39269908169872403, 5), ('RX', -1.5707963267948966, 7)]
t=25: [('MS', 1.5707963267948966, (5, 7))]
t=27: [('RX', -1.5707963267948966, 5), ('RX', -1.3744467859455334, 7)]
t=30: [('MS', 1.5707963267948966, (4, 7))]
t=34: [('RY', -0.39269908169872414, 5)]
t=37: [('MS', 1.5707963267948966, (5, 6))]
t=41: [('RY', 0.19634954084936218,

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import imageio  # type: ignore
import os
import warnings

# ------------------------------------------------------------------
#  pos_hist      : (Tₚ, Q, 2)  integer array  (already in scope)
#  scheduled_ops : list of op-lists, length = Tₛ  (already in scope)
# ------------------------------------------------------------------

Tp, Q, _ = pos_hist.shape
Ts = len(scheduled_ops)
T = min(Tp, Ts)

if Tp != Ts:
    warnings.warn(f"pos_hist has {Tp} ticks but scheduled_ops has {Ts}; "
                  f"plotting first {T} ticks only.")

frames_dir = "frames"
os.makedirs(frames_dir, exist_ok=True)

rows = int(pos_hist[:, :, 0].max()) + 1
cols = int(pos_hist[:, :, 1].max()) + 1
frame_files = []

for t in range(T):
    # ---- highlight qubits that get RX / RY this tick -------------------
    sq_highlight = {q for name, *rest in scheduled_ops[t]
                    if name in ("RX", "RY")
                    for q in rest[-1:]}

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_xlim(-0.5, cols - 0.5)
    ax.set_ylim(rows - 0.5, -0.5)
    ax.set_xticks(range(cols))
    ax.set_yticks(range(rows))
    ax.grid(True)
    ax.set_aspect("equal")

    for q in range(Q):
        r, c = pos_hist[t, q]
        if q in sq_highlight:
            ax.scatter(c, r, s=350, color="red", edgecolor="black", zorder=3)
            ax.text(c, r, str(q), color="white",
                    ha="center", va="center", weight="bold", zorder=4)
        else:
            ax.scatter(c, r, s=200, facecolors="none", edgecolors="black")
            ax.text(c, r, str(q), color="black",
                    ha="center", va="center")

    ax.set_title(f"Time step {t}")
    fname = os.path.join(frames_dir, f"frame_{t:03d}.png")
    fig.savefig(fname, dpi=100, bbox_inches="tight")
    plt.close(fig)
    frame_files.append(fname)

# Build GIF ----------------------------------------------------------
gif_path = "movement.gif"
with imageio.get_writer(gif_path, mode="I", duration=1) as writer:
    for fname in frame_files:
        writer.append_data(imageio.imread(fname))

print(f"Saved animation to {gif_path}")


C:\Users\senli\AppData\Local\Temp\ipykernel_84512\433855274.py:17: UserWarning: pos_hist has 154 ticks but scheduled_ops has 153; plotting first 153 ticks only.
  warnings.warn(f"pos_hist has {Tp} ticks but scheduled_ops has {Ts}; "
C:\Users\senli\AppData\Local\Temp\ipykernel_84512\433855274.py:62: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  writer.append_data(imageio.imread(fname))


Saved animation to movement.gif


In [4]:
from verifier import verifier
from fidelity import fidelity
import trap

pos_sequence = [
    [tuple(pos) for pos in pos_hist[t+1]]
    for t in range(len(scheduled_ops))
]

print(pos_sequence)
print(scheduled_ops)

graph = trap.create_trap_graph()

verifier(pos_sequence, scheduled_ops, graph)
fidelity(pos_sequence, scheduled_ops, graph)

[[(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (3, 1), (3, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (3, 1), (3, 2)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (3, 1), (3, 1)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (3, 1), (3, 1)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (3, 2)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (3, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (4, 1), (4, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (3, 1), (3, 3)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (3, 1), (3, 2)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (3, 1), (3, 1)], [(0, 1), (0, 3), (1, 0), (1, 4), (3, 0), (3, 4), (3, 1), (3, 1)], [(0, 1), 

0.9577935398398315

In [6]:
with open('circuit_output.txt', 'w', encoding='utf-8') as f:
    for ops in scheduled_ops:
        # repr(ops) gives you something like:
        #   [('RX', -2.35619, 7), ...]
        f.write(repr(ops) + '\n')

In [7]:
import pennylane as qml
from pennylane import numpy as np

n = 8
dev = qml.device("default.qubit", wires=n)

def custom_qft(wires):
    """Forward QFT on `wires` (no final swap) using RY and CPhase."""
    for k in range(len(wires)):
        # Hadamard via RY(π/2)
        qml.RY(np.pi/2, wires=wires[k])
        # Controlled-phase rotations
        for j in range(k+1, len(wires)):
            angle = np.pi / (2 ** (j - k))
            # PennyLane’s CPhase is allowed—internally this is a native
            # controlled-RZ, but you can decompose RZ into RX/RY if you like:
            qml.CPhase(angle, wires=[wires[k], wires[j]])

def bit_reverse_state(state):
    """Reverse the bit-order of an n-qubit statevector."""
    N = state.size
    m = int(np.log2(N))
    out = np.zeros_like(state)
    for i in range(N):
        rev = int(format(i, f'0{m}b')[::-1], 2)
        out[rev] = state[i]
    return out

@qml.qnode(dev)
def U_custom():
    custom_qft(range(n))
    return qml.state()

@qml.qnode(dev)
def U_builtin():
    qml.QFT(wires=range(n))
    return qml.state()

# Execute
state_c = U_custom()
state_c_br = bit_reverse_state(state_c)
state_b = U_builtin()

fidelity = abs(np.vdot(state_c_br, state_b))**2
print(f"State fidelity vs. built-in QFT: {fidelity:.8f}")


State fidelity vs. built-in QFT: 1.00000000


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import imageio  # type: ignore
import os, warnings

# ------------------------------------------------------------------
# pos_hist       (Tₚ, Q, 2)  : numpy array from your scheduler
# scheduled_ops  length Tₛ   : list[list[ops]] from your scheduler
# ------------------------------------------------------------------

Tp, Q, _ = pos_hist.shape
Ts = len(scheduled_ops)
T = min(Tp, Ts)

if Tp != Ts:
    warnings.warn(f"pos_hist has {Tp} ticks but scheduled_ops has {Ts}; "
                  f"animating first {T} ticks only.")

rows = int(pos_hist[:, :, 0].max()) + 1
cols = int(pos_hist[:, :, 1].max()) + 1

frames_dir = "frames"
os.makedirs(frames_dir, exist_ok=True)
frame_files = []

for t in range(T):
    # ---- red: single-qubit ops in *this* tick ---------------------------
    sq_highlight = {q for name, *rest in scheduled_ops[t]
                    if name in ("RX", "RY")
                    for q in rest[-1:]}

    # ---- green: MS gate qubits from *previous* tick ---------------------
    ms_highlight = set()
    if t > 0:
        for name, *rest in scheduled_ops[t-1]:
            if name == "MS":
                ms_highlight.update(rest[-1])   # (q1,q2)

    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_xlim(-0.5, cols - 0.5)
    ax.set_ylim(rows - 0.5, -0.5)
    ax.set_xticks(range(cols))
    ax.set_yticks(range(rows))
    ax.grid(True)
    ax.set_aspect("equal")

    for q in range(Q):
        r, c = pos_hist[t, q]
        if q in ms_highlight:
            ax.scatter(c, r, s=350, color="limegreen",
                       edgecolor="black", zorder=3)
            ax.text(c, r, str(q), color="white", weight="bold",
                    ha="center", va="center", zorder=4)
        elif q in sq_highlight:
            ax.scatter(c, r, s=350, color="red",
                       edgecolor="black", zorder=3)
            ax.text(c, r, str(q), color="white", weight="bold",
                    ha="center", va="center", zorder=4)
        else:
            ax.scatter(c, r, s=200, facecolors="none",
                       edgecolors="black", zorder=2)
            ax.text(c, r, str(q), color="black",
                    ha="center", va="center")

    ax.set_title(f"Time step {t}")
    fname = os.path.join(frames_dir, f"frame_{t:03d}.png")
    fig.savefig(fname, dpi=100, bbox_inches="tight")
    plt.close(fig)
    frame_files.append(fname)

# Build GIF ------------------------------------------------------------------
gif_path = "movement.gif"
with imageio.get_writer(gif_path, mode="I", duration=2) as writer:
    for fname in frame_files:
        writer.append_data(imageio.imread(fname))

print(f"Saved animation to {gif_path}")


C:\Users\senli\AppData\Local\Temp\ipykernel_84512\1083563131.py:16: UserWarning: pos_hist has 154 ticks but scheduled_ops has 153; animating first 153 ticks only.
  warnings.warn(f"pos_hist has {Tp} ticks but scheduled_ops has {Ts}; "
C:\Users\senli\AppData\Local\Temp\ipykernel_84512\1083563131.py:75: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  writer.append_data(imageio.imread(fname))


Saved animation to movement.gif


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import os, warnings

# --------------------------------------------------------------------
# pos_hist       (T, Q, 2) int ndarray  – already in memory
# scheduled_ops  list[list[ops]]       – same length T
# --------------------------------------------------------------------

T, Q, _ = pos_hist.shape
if len(scheduled_ops) != T:
    warnings.warn(
        f"scheduled_ops has {len(scheduled_ops)} entries "
        f"but pos_hist has {T}; using the shorter length."
    )
    T = min(T, len(scheduled_ops))

rows = int(pos_hist[:, :, 0].max()) + 1
cols = int(pos_hist[:, :, 1].max()) + 1

out_dir = "history_pngs"
os.makedirs(out_dir, exist_ok=True)

for t in range(T):
    # ── determine highlights for this frame ──────────────────────────
    sq_red = {q for name, *rest in scheduled_ops[t]
              if name in ("RX", "RY")
              for q in rest[-1:]}

    ms_green = set()
    if t > 0:
        for name, *rest in scheduled_ops[t - 1]:
            if name == "MS":
                ms_green.update(rest[-1])        # (q1,q2)

    # ── plot frame ──────────────────────────────────────────────────
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.set_title(f"Time step {t}")
    ax.set_xlim(-0.5, cols - 0.5)
    ax.set_ylim(rows - 0.5, -0.5)
    ax.set_xticks(range(cols))
    ax.set_yticks(range(rows))
    ax.grid(True)
    ax.set_aspect("equal")

    for q in range(Q):
        r, c = pos_hist[t, q]

        if q in ms_green:
            ax.scatter(c, r, s=250, color="limegreen",
                       edgecolor="black", linewidth=1, zorder=3)
            txt_color = "white"
        elif q in sq_red:
            ax.scatter(c, r, s=250, color="red",
                       edgecolor="black", linewidth=1, zorder=3)
            txt_color = "white"
        else:
            ax.scatter(c, r, s=150, facecolors="none",
                       edgecolors="black", linewidth=1)
            txt_color = "black"

        ax.text(c, r, str(q), ha="center", va="center",
                color=txt_color, fontweight="bold", fontsize=9)

    ax.invert_yaxis()  # row 0 at top (optional)

    fname = os.path.join(out_dir, f"step_{t:03d}.png")
    fig.savefig(fname, dpi=100, bbox_inches="tight")
    plt.close(fig)

print(f"Wrote {T} PNG files to '{out_dir}'")


C:\Users\senli\AppData\Local\Temp\ipykernel_84512\2543242958.py:12: UserWarning: scheduled_ops has 153 entries but pos_hist has 154; using the shorter length.
  warnings.warn(


Wrote 153 PNG files to 'history_pngs'
